### Loading Data

In [ ]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Define a list with the given ticker symbols
klci_stocks = [
    '5681.KL', '3182.KL', '1066.KL', '5285.KL', '5296.KL', '1295.KL',
    '1082.KL', '5183.KL', '7084.KL', '1155.KL', '5819.KL', '1015.KL',
    '4863.KL', '5225.KL', '4715.KL', '5347.KL', '1961.KL', '1023.KL',
    '6888.KL', '4707.KL', '4065.KL', '6947.KL', '6012.KL', '2445.KL',
    '3816.KL', '4197.KL', '4677.KL', '6033.KL', '6742.KL', '8869.KL'
]

# Display the list
print(klci_stocks)

In [ ]:

end_date = '2024-03-31'

start_date = '2022-01-01'

df = yf.download(tickers=klci_stocks,
                 start=start_date,
                 end=end_date).stack()

#set date and ticker as index
df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

## Calculate Technical Indicator



In [ ]:
!pip install pandas-ta
import pandas_ta as ta

In [ ]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

##################can try adjust length
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

## Aggregate to monthly level

In [ ]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close','daily return']]

In [ ]:
#To reduce training time and experiment with features and strategies
#convert the business-daily data to month-end frequency.

monthdata = df.unstack()[last_cols].resample('M').last().stack('ticker').dropna()

In [ ]:
monthdata

## Monthly Returns for Different Time Horizon

In [ ]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1,3,6,9]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
    
    
monthdata = monthdata.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
monthdata

## FAMA-FRENCH FIVE FACTOR


In [ ]:
factor_data = web.DataReader('Emerging_5_Factors',
                               'famafrench',
                               start='2022')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(monthdata['return_1m'])

factor_data

In [ ]:
#calculate rolling factor betas

betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1m'], 
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(6, x.shape[0]),
                                     min_nobs=len(x.columns))
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas

In [ ]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

proc_data = (monthdata.join(betas.groupby('ticker').shift()))

proc_data.loc[:, factors] = proc_data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

proc_data = proc_data.drop('adj close', axis=1)

proc_data = proc_data.dropna()

proc_data.info()


# K-Means Clustering


## Parameter selection based on Silhouette Score

In [ ]:
from sklearn.cluster import KMeans

Sum_of_squared_distances = []
K = range(1,10)
for num_clusters in K :
 kmeans = KMeans(n_clusters=num_clusters)
 kmeans.fit(proc_data)
 Sum_of_squared_distances.append(kmeans.inertia_)
plt.plot(K,Sum_of_squared_distances,'bx-')
plt.xlabel('Values of K') 
plt.ylabel('Sum of squared distances/Inertia') 
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_avg = []
K = range(2,10)
for num_clusters in K :
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(proc_data)
    cluster_labels = kmeans.labels_
    silhouette_avg.append(silhouette_score(proc_data,cluster_labels))
    
plt.plot(K,silhouette_avg,'bx-')
plt.xlabel('Values of K') 
plt.ylabel('Sum of squared distances/Inertia') 
plt.title('Silhoutte score For Optimal k')
plt.show()

As indicated by both elbow method and also the silhoutte scores (0.25-0.5 generally means a fair clustering), the suitable number of clusters is 3, but number of cluster 2 obtained a similar silhouette score

Centroids is predefined by strategy trusting the stocks with high momentum will continue perform at the next month

In [ ]:
target_rsi_values = [25,75]

initial_centroids = np.zeros((len(target_rsi_values), 16))

initial_centroids[:, 1] = target_rsi_values

initial_centroids

In [ ]:
from sklearn.cluster import KMeans


def get_clusters(df):
    df['cluster'] = KMeans(n_clusters=2,
                            random_state=42,
                            init=initial_centroids).fit(df).labels_
    
    return df

KM_random_data = proc_data.dropna().groupby('date', group_keys=False).apply(get_clusters)

KM_random_data

In [ ]:
def plot_clusters(data):

    cluster_0 = data[data['cluster']==0]
    cluster_1 = data[data['cluster']==1]


    plt.scatter(cluster_0.iloc[:,5] , cluster_0.iloc[:,1] , color = 'red', label='cluster 0')
    plt.scatter(cluster_1.iloc[:,5] , cluster_1.iloc[:,1] , color = 'green', label='cluster 1')

    
    plt.legend()
    plt.show()
    return

In [ ]:
plt.style.use('ggplot')

monthly_silhouette = []

for i in KM_random_data.index.get_level_values('date').unique().tolist():
    
    g = KM_random_data.xs(i, level=0)
    
    plt.title(f'Date {i}')
    
    plot_clusters(g)
    
    silhouette = silhouette_score(g,g['cluster'])

    monthly_silhouette.append(silhouette)
    
    


In [ ]:
from statistics import mean 


print('the mean silhouette score across months :',mean(monthly_silhouette))

In [ ]:
#Dataframe is filtered by selecting only cluster with centroid of higher RSI value
filtered_df = KM_random_data[KM_random_data['cluster']==1].copy()

filtered_df = filtered_df.reset_index(level=1)

filtered_df.index = filtered_df.index+pd.DateOffset(1)

filtered_df = filtered_df.reset_index().set_index(['date', 'ticker'])

dates = filtered_df.index.get_level_values('date').unique().tolist()

fixed_dates = {}

for d in dates:
    
    fixed_dates[d.strftime('%Y-%m-%d')] = filtered_df.xs(d, level=0).index.tolist()
    
fixed_dates

In [ ]:
#To obtain the frequency of each stock invested
stock_frequency = filtered_df.index.get_level_values('ticker').value_counts()

print(stock_frequency)

### Portfolio Optimization

In [ ]:
!pip install PyPortfolioOpt

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# define the function for optimization of weightage assigned

def optimize_weights(prices, lower_bound=0):
    
    returns = expected_returns.mean_historical_return(prices=prices,
                                                      frequency=252)
    
    cov = risk_models.sample_cov(prices=prices,
                                 frequency=252)
    
    ef = EfficientFrontier(expected_returns=returns,
                           cov_matrix=cov,
                           weight_bounds=(lower_bound, .1),
                           solver='SCS')
    
    weights = ef.max_sharpe()
    
    return ef.clean_weights()

In [ ]:
#daily data is used in optimization
#the daily data of selected stocks is obtained starts from 1 year prior 2022
stocks = KM_random_data.index.get_level_values('ticker').unique().tolist()

new_df = yf.download(tickers=stocks,
                     start=KM_random_data.index.get_level_values('date').unique()[0]-pd.DateOffset(months=12),
                     end=KM_random_data.index.get_level_values('date').unique()[-1])

new_df

In [ ]:
returns_dataframe = np.log(new_df['Adj Close']).diff()

portfolio_df = pd.DataFrame()

stock_weights=pd.DataFrame()

for start_date in fixed_dates.keys():

    try :
        end_date = (pd.to_datetime(start_date)+pd.offsets.MonthEnd(0)).strftime('%Y-%m-%d')
        
        cols = fixed_dates[start_date]
    
        #optimization requires 1year data
        optimization_start_date = (pd.to_datetime(start_date)-pd.DateOffset(months=12)).strftime('%Y-%m-%d')
            
        optimization_end_date = (pd.to_datetime(start_date)-pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    
        optimization_df = new_df[optimization_start_date:optimization_end_date]['Adj Close'][cols]

        success = False

        try:
            #optimize and find the weights for the stocks of that month
            weights = optimize_weights(prices=optimization_df,
                                   lower_bound = round(1/(len(optimization_df.columns)*2),3))
        
            weights = pd.DataFrame(weights, index=pd.Series(0))

            success = True
            
            
        except:
            #Assign equal weight to every stocks if optimization failed
            print(f'Max Sharpe Optimization failed for {start_date}, continuing with equal weight')

        if success == False:
            weight = pd.DataFrame([1/len(optimization_df.columns) for i in range(len(optimization_df.columns))],
                                    index=optimization_df.columns.tolist(),
                                    columns=pd.Series(0)).T
            
        #multiply the weight with the return for everyday
        #to calc the daily portfolio return
        temp_df = returns_dataframe[start_date:end_date]

    
        temp_df = temp_df.stack().to_frame('return').reset_index(level=0).merge(weights.stack().to_frame('weight').reset_index(level=0,drop=True),
                                                                  left_index=True,
                                                                  right_index=True).reset_index().set_index(['Date','Ticker']).unstack().stack()

    
        
        temp_df['weighted_return'] = temp_df['return']*temp_df['weight']

        stock_weights = pd.concat([stock_weights, temp_df])
    
        temp_df = temp_df.groupby(level=0)['weighted_return'].sum().to_frame('Strategy Return')
    
        portfolio_df = pd.concat([portfolio_df, temp_df],axis=0)

    except Exception as e:
        print(e)

portfolio_df = portfolio_df.drop_duplicates()

portfolio_df

In [ ]:
# to visualize the daily strategy return
portfolio_df.plot()

## Visualize Cumulative Portfolio Returns and Compare to KLCI Index

In [ ]:
klci = yf.download(tickers='^KLSE',
                   start = '2022-01-01',
                   end=dt.date.today())
klci

In [ ]:
#calculate the return by buying the overall KLCI index and holding it over the investment period
klci_ret = np.log(klci[['Adj Close']]).diff().dropna().rename({'Adj Close':'KLCI Buy&Hold'},axis=1)

klci_ret

In [ ]:
portfolio_df = portfolio_df.merge(klci_ret,
                                  left_index=True,
                                  right_index=True)

portfolio_df

In [ ]:
import matplotlib.ticker as mtick

plt.style.use('ggplot')

portfolio_cumulative_return = np.exp(np.log1p(portfolio_df).cumsum())-1

portfolio_cumulative_return[:'2024-03-29'].plot(figsize=(16,6))

plt.title('Unsupervised Learning Trading Strategy Returns Over Time')

plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1))

plt.ylabel('Return')

plt.show()